<a href="https://colab.research.google.com/github/kritika200015/Covid19_symptoms-checker/blob/main/Tpot_classification_with_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Covid-19 symptoms checker (using genetic algorithm)

Importing Lib and data

In [ ]:
import numpy as np # linear algebra
import pandas as pd
df = pd.read_csv("Cleaned-Data.csv")
dataset = df.values

In [ ]:
df.head()

Preprocessing

In [ ]:
severity_columns = df.filter(like='Severity_').columns
df['Severity_None'].replace({1:'None',0:'No'},inplace =True)
df['Severity_Mild'].replace({1:'Mild',0:'No'},inplace =True)
df['Severity_Moderate'].replace({1:'Moderate',0:'No'},inplace =True)
df['Severity_Severe'].replace({1:'Severe',0:'No'},inplace =True)

In [ ]:
df['Condition']=df[severity_columns].values.tolist()

In [ ]:
def removing(list1):
    list1 = set(list1) 
    list1.discard("No")
    a = ''.join(list1)
    return a

In [ ]:
df['Condition'] = df['Condition'].apply(removing)

In [ ]:
age_columns = df.filter(like='Age_').columns
gender_columns = df.filter(like='Gender_').columns
contact_columns = df.filter(like='Contact_').columns

In [ ]:
No_risk_age = df.groupby(['Severity_None'])[age_columns].sum()
No_risk_gender = df.groupby(['Severity_None'])[gender_columns].sum()
No_risk_contact = df.groupby(['Severity_None'])[contact_columns].sum()

In [ ]:
Low_risk_age = df.groupby(['Severity_Mild'])[age_columns].sum()
Low_risk_gender = df.groupby(['Severity_Mild'])[gender_columns].sum()
Low_risk_contact = df.groupby(['Severity_Mild'])[contact_columns].sum()

In [ ]:
Moderate_risk_age = df.groupby(['Severity_Moderate'])[age_columns].sum()
Moderate_risk_gender = df.groupby(['Severity_Moderate'])[gender_columns].sum()
Moderate_risk_contact = df.groupby(['Severity_Moderate'])[contact_columns].sum()

In [ ]:
Severe_risk_age = df.groupby(['Severity_Severe'])[age_columns].sum()
Severe_risk_gender = df.groupby(['Severity_Severe'])[gender_columns].sum()
Severe_risk_contact = df.groupby(['Severity_Severe'])[contact_columns].sum()

In [ ]:
df.drop("Country",axis=1,inplace=True)
df.drop(severity_columns,axis=1,inplace=True)
df['Symptoms_Score'] = df.iloc[:,:5].sum(axis=1) + df.iloc[:,6:10].sum(axis=1)

In [ ]:
df['Condition'].replace({'Mild':1,'None':0,'Moderate':1,'Severe':1},inplace =True)

In [ ]:
for i in df.columns:
    print("\nColumn Name:",i,"-->",df[i].unique(),"-->Unique Count",len(df[i].unique()))

In [ ]:
import tensorflow

Downcasting 

In [ ]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                      props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
            print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist


In [ ]:
df, NAlist = reduce_mem_usage(df)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

In [ ]:
!pip install tpot

Model

In [ ]:
from sklearn.model_selection import train_test_split
x=df.drop(['Condition','Age_0-9', 'Age_10-19',
       'Age_20-24', 'Age_25-59', 'Age_60+', 'Gender_Female', 'Gender_Male',
       'Gender_Transgender', 'Contact_Dont-Know', 'Contact_No', 'Contact_Yes','Symptoms_Score','None_Sympton', 'Pains','None_Experiencing'],axis=1)
y=df['Condition']

x_train, x_test, y_train, y_test = train_test_split(x,y , test_size =0.2)

In [ ]:
from tpot import TPOTClassifier


tpot_classifier = TPOTClassifier(generations= 5, population_size= 50,
                                 verbosity= 2,
                                 n_jobs = -1 , random_state = 1 , early_stop = 12,
                                 cv = 5, scoring = 'accuracy')
tpot_classifier.fit(x_train,y_train)

In [ ]:
accuracy = tpot_classifier.score(x_test,y_test)
print(accuracy)

In [ ]:
tpot_classifier.export('optimal_pipeline.py')

In [ ]:
def pipeline_suggested(df):

  exported_pipeline = BernoulliNB(alpha=0.1, fit_prior=True)
# Fix random state in exported estimator
  if hasattr(exported_pipeline, 'random_state'):
      setattr(exported_pipeline, 'random_state', 1)
 

  exported_pipeline.fit(x_train, y_train)
  print(f"Train acc: {exported_pipeline.score(x_train, y_train)}")
  print(f"Test acc: {exported_pipeline.score(x_test, y_test)}")



In [ ]:

    # Copied from optimal pipeline suggested by tpot in file "optimal_pipeline.py"
    # Initialize 
#exported_pipeline = make_pipeline(
    
    
    
    #PolynomialFeatures(degree=2, include_bias=False, interaction_only=False),
    #VarianceThreshold(threshold=0.2),
    #ZeroCount(),
    #GradientBoostingClassifier(learning_rate=1.0, max_depth=10, max_features=0.9000000000000001, min_samples_leaf=16, min_samples_split=3, n_estimators=100, subsample=0.7000000000000001)
    #)
    # Init training
from sklearn.naive_bayes import BernoulliNB
exported_pipeline = BernoulliNB(alpha=0.1, fit_prior=True)
exported_pipeline.fit(x_train, y_train)
print(f"Train acc: {exported_pipeline.score(x_train, y_train)}")
print(f"Test acc: {exported_pipeline.score(x_test, y_test)}")
   